# Econ 581
# Bill Polley
# Notes on Mathematical Economics

## Comparative Statics
Many times we are interested in how changes in one of the parameters may affect the choice variables and therefore the equilibrium outcomes. For example, how does a change in government spending effect labor and consumption choices? This branch of analysis is called "comparative statics." It is not dynamics. It is simply just as it sounds, the comparision of two static equilibria. This set of notes explores this concept by way of examples in which you can see how to solve the problem both by hand and by computer.

### A competitive equilibrium model with government
This section discusses an example of the Lagrange method in the way that it might be presented in a graduate level macroeconomics course.

Here we present a model of a very simple macroeconomy with government, but without investment or net exports. Recall from your principles of macro that $Y = C + I + G + NX$. Without investment or net exports, this model reduces to $Y = C + G$. It is a general equilibrium model, so we model a labor market and a market for final output. The representative household sells labor to the market and output is produced. The output is then purchased by government $(G)$ and the household $(C)$. There is one unit of time that the household must split between labor and leisure (as in the last section's example). The price of output is 1 (i.e. it is the numeraire good), the price of labor is the wage, $w$, and there is a productivity parameter, $z$. We will let $g$ stand for the exogenous level of government purchases. Let $\tau$ represent the lump sum tax to finance government purchases. It must be the case that $g = \tau$ (but for reasons that are best left to a macro course, we will keep them distinct in the statement of the problem).

Keeping things as simple as possible, let the production function be linear in labor supply with the productivity multiplier:

$$f(n) = zn$$

Letting $u$ be the utility from consumption and leisure and letting $v$ be the utility from government spending, the household's problem is then

$$\max_{c,l} u(c,l)+v(g)$$
subject to: $$c\leq w(1-l)-\tau$$

The Lagrangian is

$$L=u(c,l)+v(g)+\lambda(w(1-l)-\tau-c)$$

Following the example of the last section, we can see that this implies that

$$\frac{\partial u}{\partial l}=w\frac{\partial u}{\partial c}$$

The firm's problem implies that $w = z$. (In class I will explain a bit more in case this is not clear.)

A competitive equilibrium implies that
1. The household chooses consumption and leisure to maximize utility given $w$ and $\tau$.
2. The firm maximizes profit choosing n given the wage.
3. Markets clear
4. Government spending equals tax revenue

Now, let's put in a specific functional form for utility. Suppose that $u(c,l) = \log c + \log l$. Since we are not optimizing over $g$, we do not need to specify a form for $v$. As this is a fairly simple problem, we can substitute the constraint into the objective function (making use of the fact that $w = z$ and $g=\tau$ in equilibrium) and simply solve this problem:

$$\max_l \log(z(1-l)-g)+\log l+v(g)$$

For simplicity in the code, I will write $v(g)$ simply as a constant $v$ since $g$ is exogenous and thus disappears when we take the derivative. This is equivalent to assuming that $v$ is a constant function.

We can solve this symbolically using SymPy as we did in the last session.

In [1]:
from sympy import symbols,log,diff,solve,lambdify

z,leisure,g,v = symbols('z leisure g v')
# symbols is a SymPy function that will treat these variables symbolically

def utility(z,leisure,g,v):
    return log(z*(1-leisure)-g)+log(leisure)+v

def foc(z,leisure,g,v):
    return diff(utility(z,leisure,g,v),leisure)

In [2]:
print(foc(z,leisure,g,v))
sol=solve(foc(z,leisure,g,v),leisure)
print(sol)

-z/(-g + z*(1 - leisure)) + 1/leisure
[(-g + z)/(2*z)]


Thus we have $l = \frac{z-g}{2z}$. So we can now think about the comparative statics of this result. The question we would like to answer is how an increase in government affects leisure and consumption.

Simply calculate the derivative of leisure with respect to government, $\frac{dl}{dg}$.

In [4]:
print(diff((z-g)/(2*z),g))

-1/(2*z)


Consumption is $z(1 - l) - g$, so $\frac{dc}{dg}$ is

In [6]:
print(diff(z*(1-((z-g)/(2*z)))-g,g))

-1/2


A good homework or exam question would be to do this exercise with a different functional form for utility.

Try it (by hand and on computer) for $u = c^\frac{1}{2} + l^\frac{1}{2}$

Is there anything we can say about the general case without having to specify a paricular functional form for utility?  Yes!  This is where the total derivative comes in.  (You were probably wondering if we would ever have a chance to use this!)

Return to the first order condition from the original problem (and using the fact that $w = z$):

$$-z\frac{\partial u}{\partial c} + \frac{\partial u}{\partial l} = 0$$

Use the notation $u_1$ to denote the partial derivative with respect to the first argument (c), etc., and substitute the constraint in for consumption.

$$-zu_1(z(1 - l) - g, l) + u_2(z(1 - l) - g, l) = 0$$

We would like to find $\frac{dl}{dg}$ and $\frac{dc}{dg}$. To find these, totally differentiate the first order condition with respect to $l$ and $g$ (to find $\frac{dl}{dg}$) and with respect to $c$ and $g$ (to find $\frac{dc}{dg}$). We then have:

$$(z^2 u_{11} - zu_{12} - zu_{21} + u_{22})dl + (zu_{11} - u_{21})dg = 0$$

$$\frac{dl}{dg} = \frac{-zu_{11} + u_{12}}{z^2 u_{11} - 2 zu_{12} + u_{22}}$$

Can you determine the sign of this derivative?

A similar process is used to derive $\frac{dc}{dg}$:

$$\frac{dc}{dg} = \frac{zu_{12} - u_{22}}{z^2 u_{11} - 2 zu_{12} + u_{22}}$$

Can you determine the sign of this derivative?

It would not be difficult to create functions in Python to compute these derivatives numerically for a given functional form.

### Simple dynamics: A two period model

Now let us turn to another application that becomes easy to solve using the methods we have learned. We would like to introduce dynamics into our model in a simple way. This allows us to think about interest rates and discount rates as factors that influence the equilibrium. In this example, we are going to get rid of the labor market and replace it with a capital market. This keeps things simple and allows us to focus on one thing at a time.  Of course, we could have a model with both labor and capital markets, but this starts to get complicated in a hurry. We will examine that question in the section on dynamic programming.

First, a word about discounting. We discussed discounting in a previous lecture, so be sure that you are familiar with that material. In short, we need a way to be able to compare future amounts to an equivalent amount in the present. In finance and economics, the term "net present value" is often used, and sometimes shortened to simply "present value." The new idea we are introducing here is that of a "subjective" discount rate that a consumer uses in their utility calculation. This subjective discount rate is a part of the utility function and may not necessarily equal the market interest rate. This subjective discount rate becomes part of the marginal rate of intertemporal substitution which tell us how willing the consumer is to trade present consumption for future consumption.

A common formulation is to make utility in successive periods additively separable.

$$u(c_1) + \frac{1}{1+\delta}u(c_2)$$

In this utility function, $\delta$ represents the subjective discount rate. Utility from second period consumption is multiplied by $\frac{1}{1+\delta}$ (or in other words, divided by $(1+\delta)$, in order to compare it to first period utiilty.

Check your understanding:  A high value of $\delta$ would imply a ____________ (patient or impatient) consumer and a low value of $\delta$ would imply a _______________ (patient or impatient) consumer. (Fill in the blank)

Note the similarity in the structure of the problem to the labor/leisure choice problem we just did and to problems discussed over the last couple weeks. The mathematical theory is exactly the same. It's just a different economic interpretation to accompany a different problem. (Seriously, once you see this and see that so many of these problems are fundamentally alike, you are well on your way to becoming an economist!)

Now let us bring in the constraint in this two period consumption model. The choice is between consumption and saving. To keep things simple, we will assume that the consumer will be handed an endowment of $y_1$ in period 1 and $y_2$ in period 2. But the income endowment may not match their preferred consumption pattern. In other words, they may want to shift some of their consumption from one period to the other. In economic terms, shifting consumption from one period to another is referred to as saving or borrowing.

Because saving yields some interest (or borrowing implies that interest will have to be paid), the consumer's budget constraints can be expressed as follows:

$$c_1 + s = y_1$$
$$c_2 = y_2 + s(1 + r)$$

The choice of saving by the consumer is $s$, and the interest rate is $r$. If $s < 0$, then the consumer is borrowing. (Borrowing is negative saving.)

Now, observe that we can combine these two constraints into one by substituting out the saving variable. The single constraint (referred to as the "intertemporal budget constraint") is:

$$c_1 + \frac{c_2}{1 + r} = y_1 + \frac{y_2}{1 + r}$$

This equation says that the present value of consumption must be equal to the present value of income.

(It goes without saying that, technically, the present value of consumption must be less than or equal to the present value of income, but we usually focus on the equality constraint because inequality typically implies inefficiency.)

The statement of our basic problem is thus:

$$\max_{c_1,c_2} u(c_1) + \frac{1}{1+\delta}u(c_2)$$

Subject to:
$$c_1 + \frac{c_2}{1 + r} = y_1 + \frac{y_2}{1 + r}$$

Note the similarity to the problems we have already solved. The solution proceeds similarly. Work it out yourself to confirm that:

$$\frac{u'(c_2)}{(1+\delta)u'(c_1)}=\frac{1}{1+r}$$

In class exercise: Under our usual assumption that $u$ is concave, derive the relationship between first and second period consumption (which one is greater?) depending on the relationship between $\delta$ and $r$. Think about the economic intuition for this relationship.

Exercise: Solve the budget constraint for $c_2$ as a function of the other variables and substitute this function into the utility function to eliminate $c_2$. Derive $\frac{dc_1}{dr}$ by taking the total derivative. What does this say about $\frac{ds}{dr}$?